In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.datasets import fetch_20newsgroups
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
import numpy as np

In [54]:
print("Fecthing data..................................")
newsgroups_train = fetch_20newsgroups(subset='train')
newsgroups_test = fetch_20newsgroups(subset='test')
newsgroups_data = newsgroups_train.data + newsgroups_test.data

print("Data fetched successfully")
print("Turning data into Tf-IDF format................")
vectorizer = TfidfVectorizer(stop_words='english')
vectors = vectorizer.fit_transform(newsgroups_data)

print("Successfully created bag of words.")

Fecthing data..................................
Data fetched successfully
Turning data into Tf-IDF format................
Successfully created bag of words.


In [55]:
encoder = OneHotEncoder()
encoder.fit(newsgroups_train['target'].reshape(-1, 1))
train_labels = encoder.transform(newsgroups_train['target'].reshape(-1, 1))
test_labels = encoder.transform(newsgroups_test['target'].reshape(-1, 1))
labels = np.concatenate((train_labels.todense() , test_labels.todense()), axis=0) 

In [65]:
test_data = vectors[int(0.9 * vectors.shape[0]):]
train_data = vectors[:int(0.9 * vectors.shape[0])]
test_labels = labels[int(0.9 * labels.shape[0]):]
train_labels = labels[:int(0.9 * labels.shape[0])]
print(test_labels.shape)
print(test_data.shape)
print(train_labels.shape)
print(train_data.shape)

(1885, 20)
(1885, 173451)
(16961, 20)
(16961, 173451)


In [97]:
session = tf.InteractiveSession()
x = tf.placeholder(tf.float32, shape=[None, train_data.shape[1]])
y = tf.placeholder(tf.float32, shape=[None, train_labels.shape[1]])

w = tf.Variable(tf.zeros(shape=[train_data.shape[1],train_labels.shape[1]]))
b = tf.Variable(tf.zeros(shape=[train_labels.shape[1]]))

session.run(tf.global_variables_initializer())

In [110]:
y_ = tf.matmul(x,w) + b

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=y_))
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

batch_size = 10
for index in range(2000):
    data = vectors[index * batch_size: (index + 1) * batch_size]
    data = data.todense()
    label = labels[index * batch_size: (index + 1) * batch_size]
    train_step.run(feed_dict={x:np.array(data), y:np.array(label)})

In [111]:
correct_prediction = tf.equal(tf.argmax(y_, 1), tf.arg_max(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print("Accuracy:", accuracy.eval(feed_dict={x:test_data.todense(), y:test_labels}))

Accuracy: 0.84456235
